### 테스트케이스를 줄이기 위해 실험셋을 간소화함

#### 간소화 방식
1. 그룹 내 같은 타입의 인스턴스 중 가장 작은 인스턴스 선택
    * m5.large, m5.4xlarge, m5.16xlarge가 있다면 m5.large만 선택함
2. 오리지널 타입(m5, t3와 같은)과 a, g, i 등 프로세서를 나타내는 심볼을 가진 타입들을 선택
    * m5a, m5i가 있다면 둘 다 실험세트에 포함
3. 오리지널 타입이 없는 경우 나머지 심볼을 가진 인스턴스 선택
    * m5가 없는데 m5d가 있다면 m5d를 실험세트에 포함
    * m5가 없고 m5d가 추가된 상황이라면 m5dn, m5b 등의 타입은 포함하지 않음
4. 위 과정을 거치고 나서 인스턴스 수가 2개 미만인 그룹 제거
    * 마이그레이션 타겟이 동일한 인스턴스가 될 것으로 실험 의미가 없음


In [119]:
# 코드 자동완성 허용
%config Completer.use_jedi = False

# cell 너비 확장
from IPython.display import display, HTML

display(
    HTML(
        data="""
            <style>
                div#notebook-container      { width: 99%}
                div#menubar-container       { width: 99%}
                div#maintoolbar-container   { width: 99%}
            </style>
        """
    )
)

In [62]:
# import library
from IPython.display import display
import pandas as pd
pd.set_option('display.max_columns', None) # Unlimiting Output Length
pd.set_option('display.max_row', None) # Unlimiting Output Length
pd.set_option('display.max_colwidth', None) # Unlimiting Output Column Width Length

import gspread as gs
from gspread_formatting import *

import re

In [5]:
# read google spread sheet(core features)

gc = gs.service_account(filename='../secure-outpost-380004-8d45b1504f3e.json')

sheet = gc.open('CPU Feature Visualization').worksheet('groupby aws(core)')
df = pd.DataFrame(sheet.get_all_records())

df = df['feature groups']
df

0                                                                                                                                                                                                                                                                                                                                                                                                                          m5a.large, r5a.large, r5ad.large, m5ad.large, t3a.medium, t3a.xlarge, m5a.xlarge, r5a.xlarge, m5ad.xlarge, r5ad.xlarge, t3a.2xlarge, m5a.2xlarge, r5a.2xlarge, r5ad.2xlarge, m5ad.2xlarge, m5a.4xlarge, m5ad.4xlarge, r5a.4xlarge, r5ad.4xlarge, r5ad.8xlarge, r5a.8xlarge, m5a.8xlarge, m5ad.8xlarge, m5a.16xlarge, r5a.16xlarge, r5ad.16xlarge, m5ad.16xlarge
1                                                                                                                                                                                                                                            

In [73]:
# df to list
groups = []

for i in range(len(df)):
    groups.append(df.iloc[i].split(', '))

print(groups)

[['m5a.large', 'r5a.large', 'r5ad.large', 'm5ad.large', 't3a.medium', 't3a.xlarge', 'm5a.xlarge', 'r5a.xlarge', 'm5ad.xlarge', 'r5ad.xlarge', 't3a.2xlarge', 'm5a.2xlarge', 'r5a.2xlarge', 'r5ad.2xlarge', 'm5ad.2xlarge', 'm5a.4xlarge', 'm5ad.4xlarge', 'r5a.4xlarge', 'r5ad.4xlarge', 'r5ad.8xlarge', 'r5a.8xlarge', 'm5a.8xlarge', 'm5ad.8xlarge', 'm5a.16xlarge', 'r5a.16xlarge', 'r5ad.16xlarge', 'm5ad.16xlarge'], ['c5ad.large', 'c5a.large', 'c5ad.xlarge', 'c5a.xlarge', 'c5ad.2xlarge', 'c5a.2xlarge', 'c5ad.4xlarge', 'c5a.4xlarge', 'c5a.8xlarge', 'c5ad.8xlarge', 'c5ad.16xlarge', 'c5a.16xlarge'], ['t2.large', 'c4.large', 'm4.large', 'c4.xlarge', 'm4.xlarge', 'm4.2xlarge', 'c4.2xlarge', 't2.2xlarge', 'm4.4xlarge', 'c4.4xlarge'], ['r6a.large', 'c6a.large', 'm6a.large', 'c6a.xlarge', 'r6a.xlarge', 'm6a.xlarge', 'r6a.2xlarge', 'c6a.2xlarge', 'm6a.2xlarge', 'r6a.4xlarge', 'c6a.4xlarge', 'm6a.4xlarge', 'c6a.8xlarge', 'r6a.8xlarge', 'm6a.8xlarge', 'c6a.16xlarge', 'r6a.16xlarge', 'm6a.16xlarge'], ['c6in

In [131]:
sizelist = ['micro', 'small', 'medium', 'large', 'xlarge', '2xlarge', '4xlarge', '8xlarge', '16xlarge']
newgroup = []
newgroups = []
typelist = []
unduplicate = []

# select the smallest instance size
for group in groups:
    # extract to instance type ex) m5.large -> m5
    for instance in group:
        typelist.append(instance[:instance.index('.')])

    # delete duplicate
    unduplicate = list(set(typelist))

    # extract to smallest instance for unduplicate typelist
    for i in range(len(unduplicate)):
        pattern = '^' + unduplicate[i] + '\.'
        result = [instance for instance in group if re.match(pattern, instance)]

        smallest_size = len(sizelist) - 1
        smallest = 0
        for j in range(len(result)):
            size = result[j][result[j].index('.') + 1:]
            if sizelist.index(size) < smallest_size:
                smallest_size = sizelist.index(size)
                smallest = j
        # print(result)
        newgroup.append(result[smallest])
    print(newgroup)
    newgroups.append(newgroup)
    newgroup = []
    typelist = []
    unduplicate = []


['m5a.large', 'r5a.large', 'm5ad.large', 't3a.medium', 'r5ad.large']
['c5ad.large', 'c5a.large']
['c4.large', 't2.large', 'm4.large']
['c6a.large', 'm6a.large', 'r6a.large']
['r6i.large', 'm6idn.large', 'm6id.large', 'm6i.large', 'r6in.large', 'i4i.large', 'r6id.large', 'c6in.large', 'm6in.large', 'c6id.large', 'c6i.large', 'r6idn.large']
['m5.xlarge', 'r5.large', 'r5d.large', 'm5d.large', 'i3en.large', 't3.medium', 'c5.large']
['r5b.large', 'r5dn.large', 'm5n.large', 'm5dn.large', 'm5zn.large', 'r5n.large']
['r4.large', 'i3.large']
['m5.large', 'r5d.4xlarge', 'z1d.large', 'i3en.2xlarge', 'm5d.4xlarge', 'c5n.large', 't3.2xlarge', 'c5d.large']
['r6i.16xlarge', 'm6idn.16xlarge', 'm6id.16xlarge', 'm6i.16xlarge', 'i4i.16xlarge', 'r6in.16xlarge', 'r6id.16xlarge', 'c6in.16xlarge', 'm6in.16xlarge', 'c6id.16xlarge', 'c6i.16xlarge', 'r6idn.16xlarge']
['m4.16xlarge']
['r4.8xlarge', 'i3.8xlarge']


In [195]:
simplized_group = []
# Simplify instance types by processor type
for group in newgroups:
    tempgroup = []
    print(group)
    for instance in group:
        instance_group_name = instance[:instance.index('.')]

        if len(instance_group_name) == 2:
            tempgroup.append(instance)
            continue

        if len(instance_group_name) == 3 and instance_group_name[2] in {'a', 'i', 'g'}:
            tempgroup.append(instance)
            continue
    
    for instance in group:
        instance_group_name = instance[:instance.index('.')]

        if instance in tempgroup:
            continue
    
        if len(instance_group_name) > 3 and instance_group_name[2] in {'a', 'i', 'g'}:
            if instance[:3] + instance[instance.index('.'):] not in tempgroup:
                tempgroup.append(instance)
        else:
            pattern = '^' + instance[:2] + '.'
            result = [instance for instance in tempgroup if re.match(pattern, instance)]
            if(len(result) == 0):
                tempgroup.append(instance)
    simplized_group.append(tempgroup)

final_experiment_set = []
deleted_index = []
# Remove a group with 1 instance
i = 0
for sublist in simplized_group:
    if len(sublist) > 1:
        final_experiment_set.append(sublist)
    else:
        deleted_index.append(i)
    i += 1
        

print()
for group in final_experiment_set:
    print(group)

print(deleted_index)



['m5a.large', 'm5ad.large', 'r5a.large', 'r5ad.large', 't3a.medium']
['c5a.large', 'c5ad.large']
['c4.large', 'm4.large', 't2.large']
['c6a.large', 'm6a.large', 'r6a.large']
['c6i.large', 'c6id.large', 'c6in.large', 'i4i.large', 'm6i.large', 'm6id.large', 'm6idn.large', 'm6in.large', 'r6i.large', 'r6id.large', 'r6idn.large', 'r6in.large']
['c5.large', 'i3en.large', 'm5.xlarge', 'm5d.large', 'r5.large', 'r5d.large', 't3.medium']
['m5dn.large', 'm5n.large', 'm5zn.large', 'r5b.large', 'r5dn.large', 'r5n.large']
['i3.large', 'r4.large']
['c5d.large', 'c5n.large', 'i3en.2xlarge', 'm5.large', 'm5d.4xlarge', 'r5d.4xlarge', 't3.2xlarge', 'z1d.large']
['c6i.16xlarge', 'c6id.16xlarge', 'c6in.16xlarge', 'i4i.16xlarge', 'm6i.16xlarge', 'm6id.16xlarge', 'm6idn.16xlarge', 'm6in.16xlarge', 'r6i.16xlarge', 'r6id.16xlarge', 'r6idn.16xlarge', 'r6in.16xlarge']
['m4.16xlarge']
['i3.8xlarge', 'r4.8xlarge']

['m5a.large', 'r5a.large', 't3a.medium']
['c4.large', 'm4.large', 't2.large']
['c6a.large', 'm6a.lar

In [219]:
# read google spread sheet(core features)

gc = gs.service_account(filename='../secure-outpost-380004-8d45b1504f3e.json')

sheet = gc.open('CPU Feature Visualization').worksheet('groupby aws(core)')
df = pd.DataFrame(sheet.get_all_records())

for index in sorted(deleted_index, reverse = True):
    df = df.drop(index)

df = df.reset_index()
df.drop('index', axis=1, inplace=True)
for i in range(len(final_experiment_set)):
    df.at[i, 'feature groups'] = ', '.join(final_experiment_set[i])

# write google spread sheet1(core features)

gc = gs.service_account(filename='../secure-outpost-380004-8d45b1504f3e.json')

sheet = gc.open('CPU Feature Visualization').worksheet('simplized aws group(core)')
sheet.clear() # 이전 데이터 삭제
sheet.update([df.columns.values.tolist()] + df.values.tolist())

format_cell = cellFormat(
    verticalAlignment='MIDDLE', 
    wrapStrategy='OVERFLOW_CELL', 
    textFormat=textFormat(fontSize=10)
)

format_cell_range(sheet, '1:500', format_cell)

{'spreadsheetId': '17V0eoAl-4STj_KoQ5SOf2wRohbt6LxovyQ2HDvIxt-c',
 'replies': [{}]}